In [1]:
import requests
from collections import OrderedDict
from ipy_table import make_table
import vcr
cassette = vcr.use_cassette('sectors_data.yml')
cassette.__enter__()

In [2]:
# From https://docs.google.com/spreadsheets/d/1U79vkTIZ8U0I_N0QtOPZ6ajFkoPH-Ym2yEkdYPPPW74/edit#gid=0
ag_sector_codes = [
'12240', 
'14040', 
'311', 
'31110', 
'31120', 
'31130', 
'31140', 
'31150', 
'31161', 
'31162', 
'31163', 
'31164', 
'31165', 
'31166', 
'31181', 
'31182', 
'31191', 
'31192', 
'31193', 
'31194', 
'31195', 
'312', 
'31210', 
'31220', 
'31261', 
'31281', 
'31282', 
'31291', 
'313', 
'31310', 
'31320', 
'31381', 
'31382', 
'31391', 
'32161', 
'32162', 
'32165', 
'43040', 
'520', 
'52010', 
'72040', 
]

In [3]:
codelist_values = requests.get('http://dashboard.iatistandard.org/stats/current/inverted-publisher/codelist_values.json').json(object_pairs_hook=OrderedDict)

In [4]:
publishers_ckan = requests.get('http://iatiregistry.org/api/3/action/organization_list?all_fields=1').json()
publisher_names = {publisher['name']:publisher['display_name'] for publisher in publishers_ckan['result']}

In [5]:
publisher_activities = requests.get('http://dashboard.iatistandard.org/stats/current/inverted-publisher/activities.json').json()

In [6]:
sector_code_lookup = codelist_values['.//sector/@code']
make_table(
    [
        (sector_code,publisher,count)
            for sector_code in ag_sector_codes
            for publisher,count
            in sector_code_lookup[sector_code].items()
    ])

12240,afd,5
12240,amrefnl,1
12240,ausgov,29
12240,be-dgd,3
12240,bmgf,165
12240,bmz,24
12240,cafod,12
12240,cordaid,10
12240,danida,12
12240,dfid,299
12240,ec-devco,100


In [7]:
publishers = sorted(set(publisher for sector_code in ag_sector_codes for publisher,_ in sector_code_lookup[sector_code].items()))
make_table(
    [['Registry Slug', 'Publisher Name on Registry', 'Total Activities'] + ag_sector_codes] + [
        [publisher, publisher_names[publisher], publisher_activities[publisher]]+[
            sector_code_lookup[sector_code].get(publisher, '')
            for sector_code in ag_sector_codes
        ]
        for publisher in publishers
    ])

Registry Slug,Publisher Name on Registry,Total Activities,12240,14040,311,31110,31120,31130,31140,31150,31161,31162,31163,31164,31165,31166,31181,31182,31191,31192,31193,31194,31195,312,31210,31220,31261,31281,31282,31291,313,31310,31320,31381,31382,31391,32161,32162,32165,43040,520,52010,72040
aai,ActionAid International,78,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,3,
abt,Abt Associates Inc,4,,,,,2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
adpp,Ajuda de Desenvolvimento de Povo para Povo (ADPP),1,,,,,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
af,Adaptation Fund,30,,,5,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,5,,5,
afd,Agence Française de Développement,1249,5,,,9,46,,8,,12,5,1,,2,2,3,,1,1,1,3,1,,3,2,2,,,,,,,,,,1,,,18,,7,
afdb,African Development Bank,3798,,,,540,,,81,,18,10,64,,,,,,,,,,,,,18,,,,,,,54,,,,34,,,64,,,
aiddata,AidData,24319,,,1684,,1,,26,,,,,,,,,,,,,,,230,,,,,,,26,,,,,,,,,,26,1,
amrefnl,Amref Flying Doctors,15,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ausgov,Australia - Department of Foreign Affairs and Trade,10398,29,4,,90,34,12,11,3,16,9,12,,,2,15,523,1,9,5,,51,,12,4,,2,32,1,,54,19,2,46,1,1,,,21,,96,96
be-dgd,Belgian Development Cooperation,1001,3,1,,19,40,1,3,2,10,1,8,1,1,2,3,2,9,,1,35,5,,3,,1,,,,,,4,1,1,,,,,33,,,30


In [8]:
cassette.__exit__()